In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten,  MaxPooling2D,  Input


KeyboardInterrupt: 

In [ ]:

BASE_DIR = r'C:\Users\prakh\Desktop\archive\UTKFace'
#labels = age, gender, ethnicity
image_paths= []
age_labels= []
gender_labels=[]

for filename in tqdm(os.listdir(BASE_DIR)):
    image_path = os.path.join(BASE_DIR, filename)
    temp = filename.split('_')
    age=int(temp[0])
    gender=int(temp[1])
    image_paths.append(image_path)
    age_labels.append(age)
    gender_labels.append(gender)

In [ ]:
df=pd.DataFrame()
df['image'],  df['age'], df['gender'] = image_paths, age_labels, gender_labels
print(df)
    

In [ ]:
gender_dict={0:'Male',1:'Female'}

EDA

In [ ]:
from PIL import Image
img=Image.open(df['image'][0])
plt.axis('off')
plt.imshow(img)

In [ ]:
sns.distplot(df['age'])

In [ ]:
import random

plt.figure(figsize=(20, 20))

# Select a random subset of 25 rows from the DataFrame
files = df.sample(n=25, random_state=42)  # You can adjust the random_state for reproducibility

# Counter variable for subplot index
subplot_index = 1

for _, (file, age, gender) in files.iterrows():
    plt.subplot(5, 5, subplot_index)
    
    img = load_img(file)
    img = np.array(img)
    
    plt.imshow(img)
    plt.title(f"Age: {age} Gender: {gender_dict[gender]}")

    # Increment subplot index
    subplot_index += 1
    plt.axis('off')


Feature extraction

In [ ]:
from PIL import Image

def extract_features(images):
    features=[]
    for image in tqdm(images):
        img=load_img(image)
        img=img.resize((128,128),Image.Resampling.LANCZOS)
        img=np.array(img)
        features.append(img)
        
    features=np.array(features)
    return features       
        

In [ ]:
x=extract_features(df['image'])

In [ ]:
x.shape

In [ ]:
#normalize the image
x=x/255.0

In [ ]:
y_gender=np.array(df['gender'])
y_age=np.array(df['age'])

In [ ]:
input_shape=(128,128,3)

ModelCreation

In [ ]:
inputs = Input((input_shape))
#convolutional
conv_1 = Conv2D(32, kernel_size=(3,3),activation='relu') (inputs)
maxp_1= MaxPooling2D(pool_size=(2,2)) (conv_1)
conv_2 = Conv2D(64, kernel_size=(3,3),activation='relu') (maxp_1)
maxp_2= MaxPooling2D(pool_size=(2,2)) (conv_2)
conv_3 = Conv2D(128, kernel_size=(3,3),activation='relu') (maxp_2)
maxp_3= MaxPooling2D(pool_size=(2,2)) (conv_3)
conv_4 = Conv2D(256, kernel_size=(3,3),activation='relu') (maxp_3)
maxp_4= MaxPooling2D(pool_size=(2,2)) (conv_4)
#flatten

flatten=Flatten() (maxp_4)
#dense layers

dense_1 = Dense(256,activation='relu') (flatten)
dense_2 = Dense(256,activation='relu') (flatten)

dropout_1 = Dropout(0.3) (dense_1)
dropout_2 = Dropout(0.3) (dense_2)

output_1= Dense(1,activation='sigmoid',name='gender_out') (dropout_1)
output_2= Dense(1,activation='relu',name='age_out') (dropout_2)

model = Model(inputs=[inputs], outputs=[output_1,output_2])

model.compile(loss=['binary_crossentropy','mae'],optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model)


In [ ]:
#train model
history=model.fit(x=x,y=[y_gender,y_age], batch_size=32, epochs=30,validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

image_index = 1006
print("Original Gender:", gender_dict[y_gender[image_index]], "Original Age:", y_age[image_index])

# Assuming X contains color images with shape (height, width, channels)
original_image = x[image_index]

# Assuming your model expects color images with shape (height, width, channels)
pred = model.predict(original_image.reshape(1, 128, 128, 3))  # Adjust the number of channels accordingly

# Assuming your model outputs gender and age predictions
pred_gender = gender_dict[int(round(pred[0][0].item()))]
pred_age = int(round(pred[1][0].item()))

print("Predicted Gender:", pred_gender, "Predicted Age:", pred_age)

# Display the original color image
plt.axis('off')
plt.imshow(original_image)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import cv2  # Assuming you are using OpenCV to read color images

image_index = 3000
print("Original Gender:", gender_dict[y_gender[image_index]], "Original Age:", y_age[image_index])

# Assuming X contains color images with shape (height, width, channels)
original_image = x[image_index]

# Assuming your model expects color images with shape (height, width, channels)
pred = model.predict(original_image.reshape(1, 128, 128, 3))  # Adjust the number of channels accordingly

# Assuming your model outputs gender and age predictions
pred_gender = gender_dict[round(pred[0][0])]
pred_age = round(pred[1][0])

print("Predicted Gender:", pred_gender, "Predicted Age:", pred_age)

# Display the original color image
plt.axis('off')
plt.imshow(original_image)
plt.show()


In [ ]:
#model.save('age_gender_model(final).h5')


In [ ]:

# Real-time age prediction from the camera
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# Load the trained model from the file
loaded_model = load_model('age_gender_model(final).h5')
# ... (your existing code)

# Function to preprocess the input image
def preprocess_image(image):
    img = cv2.resize(image, (128, 128))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img


# Function to predict age and gender from the camera
def predict_age_gender_from_camera(model):
    # Open the default camera (usually the built-in webcam)
    cap = cv2.VideoCapture(0)

    while True:
        # Capture a frame from the camera
        ret, frame = cap.read()

        # Preprocess the frame
        input_image = preprocess_image(frame)

        # Make predictions using the loaded model
        gender_pred, age_pred = model.predict(input_image)

        # Extract the age and gender predictions
        predicted_age = int(age_pred[0][0])
        predicted_gender = "Male" if gender_pred[0][0] < 0.5 else "Female"

        # Display the frame with predicted age and gender
        cv2.putText(frame, f"Predicted Age: {predicted_age} Gender: {predicted_gender}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("Real-time Age and Gender Prediction", frame)

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera
    cap.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()

# Call the function to predict age and gender from the camera in real-time using the loaded model
predict_age_gender_from_camera(loaded_model)


In [ ]:
# plot results for gender
acc = history.history['gender_out_accuracy']
val_acc = history.history['val_gender_out_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()

loss = history.history['gender_out_loss']
val_loss = history.history['val_gender_out_loss']

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()

In [ ]:

# # Real-time age prediction from the camera
# import cv2
# import numpy as np
# from keras.models import load_model
# from keras.preprocessing.image import img_to_array

# # Load the trained model from the file
loaded_model = load_model('age_gender_model(30e).h5')

image_index = 148
print("Original Gender:", gender_dict[y_gender[image_index]], "Original Age:", y_age[image_index])

# Add a batch dimension to the input
input_image = x[image_index].reshape(1, 128, 128, 3)

# Predict from the model
pred = loaded_model.predict(input_image)

# Access the predicted gender and age
pred_gender = gender_dict[int(round(pred[0][0][0]))]
pred_age = int(round(pred[1][0][0]))

print("Predicted Gender:", pred_gender, "Predicted Age:", pred_age)

# Reshape the original image to its original shape
original_image = x[image_index].reshape(128, 128, 3)

plt.axis('off')
plt.imshow(original_image, cmap='gray');



In [ ]:
from keras.models import load_model
import matplotlib.pyplot as plt
# Load the saved model
loaded_model = load_model(r'C:\Users\prakh\Desktop\Coding\Deep learnign\opencv\age_gender_model(final).h5')

# Assuming 'x', 'y_gender', and 'y_age' are your training data
evaluation = loaded_model.evaluate(x=x, y=[y_gender, y_age], batch_size=32)

# Display the evaluation results
print("Evaluation Loss: ", evaluation[0])
print("Gender Accuracy: ", evaluation[4])  # Index 4 corresponds to gender_out_accuracy
print("Age Accuracy: ", evaluation[5])     # Index 5 corresponds to age_out_accuracy
